In [1]:
import pandas as pd

In [2]:
melbourne_data = pd.read_csv("melb_data.csv")

In [3]:
predictor_columns = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']

In [4]:
x = melbourne_data[predictor_columns]
y = melbourne_data.Price # our target - what we try to predict

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y)

In [6]:
from sklearn.impute import SimpleImputer
column_imputer = SimpleImputer()
imputed_x_train = pd.DataFrame(column_imputer.fit_transform(x_train))
imputed_x_test = pd.DataFrame(column_imputer.transform(x_test))
# now put these column names back into these imputed data as imputation leaves column names removed
imputed_x_train.columns = x_train.columns
imputed_x_test.columns = x_test.columns
x_train = imputed_x_train
x_test = imputed_x_test

In [7]:
categorical_variables_names = (x_train.dtypes == 'object') # selects the other data type
categorical_variables_names = list(categorical_variables_names[categorical_variables_names].index)

In [8]:
from sklearn.preprocessing import OrdinalEncoder

OE_x_train = x_train.copy()
OE_x_test = x_test.copy()
OE = OrdinalEncoder()

OE_x_train[categorical_variables_names] = OE.fit_transform(OE_x_train[categorical_variables_names])
OE_x_test[categorical_variables_names] = OE.transform(OE_x_test[categorical_variables_names])
x_train = OE_x_train
x_test = OE_x_test

XGBoost Modelling:
Naive Model -> Make Predictions -> Calculate Loss (MAE) -> Train New Model -> Add new model to the ensemble -> repeat from step 2

In [ ]:
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

XGB_Regressor = XGBRegressor(n_estimators=1000, learning_rate=0.05)
XGB_Regressor.fit(x_train, y_train, early_stopping_rounds=5, eval_set = [(x_test, y_test)], verbose = False)
predictions = XGB_Regressor.predict(x_test)
MAE_result = mean_absolute_error(y_test, predictions)
print("The MAE when using XBG Boost: ", MAE_result)

## Data Leakage is Dangerous
Data Leakage is when your training data contains details about the target which may not be available in real life application.

There are two types of Data Leakage:

Target Leakage:—When Predictors include information that will not be available in real life scenario or even have features ordered differently to real life data. For example we want to predict who has a disease, we have x,y,z data and also a further column which states takes X medicine. This is an event that will happen post-disease diagnosis, henceforth will not be available in prediction stage, thereby should be excluded.
Train-Test Contamination: - When preprocessing is accidentally applied before train_test_split, resulting in the pre-processing to make the model behave good although it is not tested with the proper data we intended. To fix this, use pipelines.



Solution is to drop the potential leakage columns, better safe than sorry
## Also ALTHOUGH the MAE score may look good with the leakage columns it will perform poorly in real life

In [ ]:
`